In [ ]:
import pandas as pd           

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from scipy import integrate
from scipy import stats
import random

from scipy.stats import spearmanr, pearsonr
from latex_format import float2latex


In [ ]:
from selection_coefficient import Problem_M3, get_ODE_solution

In [ ]:
### Update dependent parameters according to input
import os
import os.path
from os import path

## create export directory if necessary
## foldernames for output plots/lists produced in this notebook
import os
FIG_DIR = f'./figures/M3_growthcurves/'
os.makedirs(FIG_DIR, exist_ok=True)


In [ ]:


### execute script to load modules here
exec(open('setup_aesthetics.py').read())

In [ ]:
DATASET_COLOR = 'darkorange'
palette = {'wild-type':'orange', 'knockout': 'dimgrey', 'wild-type median':'navy'}

### Load wild-type traits

In [ ]:
INDEX_COL = [0,1,2,3,4]
list_na_representations = ['not_present', 'failed_to_compute']

In [ ]:
PCWS_TRAITS_WARRINGER = './output/df_M3_traits.csv'
df_warringer = pd.read_csv(PCWS_TRAITS_WARRINGER, header = 0, index_col= INDEX_COL,\
                                  float_precision=None, na_values=list_na_representations)


In [ ]:
### define default wild_type
df_wildtypes = df_warringer[df_warringer['is_wildtype']==True]

WILDTYPE = df_wildtypes.median(axis = 0)

### Load mutant data (averaged)

In [ ]:

PCWS_TRAITS_WARRINGER_AVERAGED = './output/df_M3_traits_averaged.csv'
df_averaged = pd.read_csv(PCWS_TRAITS_WARRINGER_AVERAGED, header = 0, float_precision=None)

In [ ]:
### assign wild-type label
def is_wildtype(row):
    genotype = row['genotype']
    
    if genotype == 'BY4741':
        return True
    else:
        return False
    

row = df_averaged.iloc[0]
is_wildtype(row)

In [ ]:
df_averaged['is_wildtype'] = df_averaged.apply(is_wildtype, axis =1)

In [ ]:
### append mutant values (averaged) to set of individual wild-type strains
df_knockouts = df_averaged[~df_averaged['is_wildtype']]


### Set units of time

In [ ]:
df_knockouts['gmax'] = df_knockouts['gmax']*60 # change units to growth rate per hour
df_knockouts['lag']  = df_knockouts['lag']/60 # change units to hour



In [ ]:
WILDTYPE['gmax'] = WILDTYPE['gmax']*60 # change units to growth rate per hour
WILDTYPE['lag']  = WILDTYPE['lag']/60 # change units to hour

### Define initial conditions

In [ ]:
### set initial resource concentrations

CONCENTRATION_GLUCOSE = 20/180 * 1e3 # concentrations are recored  in milliMolar, to match the units of yield
print(CONCENTRATION_GLUCOSE)

In [ ]:
### define default initial_OD
OD_START = 0.05  #df_warringer['od_start'].median()

### compare to initial OD in the monoculture cycles
fig, ax = plt.subplots(figsize = (FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET))

ax = df_warringer['od_start'].hist(bins=41, color = DATASET_COLOR, alpha = 0.6, log = True, rasterized = True)


ax.axvline(OD_START, color = 'tab:red', label = f'median value: $N_0={OD_START:.3f}$')
ax.legend()
ax.set_xlabel('initial OD')
ax.set_ylabel('no. growth curves')

### Plot growth curves

In [ ]:
from selection_coefficient import Problem_M3, get_ODE_solution

In [ ]:
solver_params = {'t_final': 20,
 'timestep': 0.1,
 'adaptive_timewindow': False,
 'rtol': 1e-08,
 'atol': 1e-12,
 'scoeff_atol': 1e-08,
 'scoeff_rtol': 1e-06}

### Plot pairwise competition strain growth-curves using simulation

In [ ]:
### choose a nice strain
is_longer_lag = df_knockouts['lag'] > 1.5*WILDTYPE['lag']
is_faster_growth = df_knockouts['gmax'] > 1.1*WILDTYPE['gmax']

is_candidate = is_longer_lag & is_faster_growth
list_candidates = np.arange(0,len(df_knockouts))[is_candidate]
print(list_candidates)

In [ ]:
index = 35
l2, g2, Y2 = df_knockouts.loc[index, 'lag'], df_knockouts.loc[index, 'gmax'], df_knockouts.loc[index, 'yield']
l1, g1, Y1 = WILDTYPE['lag'], WILDTYPE['gmax'], WILDTYPE['yield']

strain_params = {'lam':[l1,l2], 'g':[g1,g2], 'Y':[Y1,Y2]}
initial_conditions = {'R_0': CONCENTRATION_GLUCOSE, 'N_0': OD_START}
problem = Problem_M3(**strain_params, **initial_conditions, x =0.5)

In [ ]:
sol = get_ODE_solution(problem, **solver_params)

In [ ]:
fig, ax = plt.subplots(figsize = (0.7*FIGWIDTH_TRIPLET,0.7*FIGHEIGHT_TRIPLET))

ax.plot(sol.t,sol.y[0], color = palette['wild-type median'], lw = 3, zorder =54)
ax.plot(sol.t,sol.y[1], color = palette['knockout'], lw = 3)
ax.set_yscale('log')
ax.set_ylabel('absolute abundance [OD]')
ax.set_xlabel('time [hours]')
ax.set_xlim(0,sol.t[-1])
ax.set_ylim(ymax = 20)

fig.savefig(FIG_DIR + f"pairwise_competition_{index}.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

In [ ]:
## store axis boundaries
ymin,ymax = ax.get_ylim()

### Plot single strain growth-curves using simulation

In [ ]:
problem = Problem_M3(**strain_params, **initial_conditions, x =1.0)

In [ ]:
sol = get_ODE_solution(problem, **solver_params)

In [ ]:
fig, ax = plt.subplots(figsize = (0.7*FIGWIDTH_TRIPLET,0.7*FIGHEIGHT_TRIPLET))

ax.plot(sol.t,sol.y[1], color = palette['knockout'], lw = 3)
ax.set_ylim(ymin,ymax)
ax.set_yscale('log')
ax.set_ylabel('absolute abundance [OD]')
ax.set_xlabel('time [hours]')
ax.set_xlim(0,sol.t[-1])

fig.savefig(FIG_DIR + f"knockout_averaged.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

### Load raw growth curve data

In [ ]:
exec(open('setup_plateau_finder_warringer2003.py').read())

In [ ]:
gene = df_knockouts.loc[index, 'genotype']

In [ ]:
## pick  curves to plot

list_to_plot = []
for name in df_trimmed.index:
    if name[0]==gene:
        list_to_plot.append(name)

# list_to_plot = [0]

In [ ]:
### plot growth curve
fig, ax = plt.subplots(figsize = (0.7*FIGWIDTH_TRIPLET,0.7* FIGHEIGHT_TRIPLET))

# plot original growth curves
for name in list_to_plot:

    t_trimmed, od_trimmed = get_excess_od_timeseries_before_trim(name)
    ax.scatter(t_trimmed/60, od_trimmed, color = 'tab:green', marker = 'o', s = 6)
    
# plot mean growth curve
ax.plot(sol.t,sol.y[1], color = palette['knockout'], lw = 3, zorder = 3)

ax.set_yscale('log')
ax.set_xlabel('time [hours]')
ax.set_ylabel('absolute abundance [OD]')

ax.set_xlim(0,sol.t[-1])
ax.set_ylim(ymin,ymax)

fig.savefig(FIG_DIR + f"knockout_averaged_with_raw_data.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

### Plot wildtype growth-curves using simulation

In [ ]:
problem = Problem_M3(**strain_params, **initial_conditions, x =0.0)

In [ ]:
sol = get_ODE_solution(problem, **solver_params)

In [ ]:
fig, ax = plt.subplots(figsize = (0.7*FIGWIDTH_TRIPLET,0.7*FIGHEIGHT_TRIPLET))

ax.plot(sol.t,sol.y[0], color = palette['wild-type median'], lw = 3)
ax.set_ylim(ymin,ymax)
ax.set_yscale('log')
ax.set_ylabel('absolute abundance [OD]')
ax.set_xlabel('time [hours]')
ax.set_xlim(0,sol.t[-1])

fig.savefig(FIG_DIR + f"wildtype_averaged.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

### Load raw growth curve data

In [ ]:
exec(open('setup_plateau_finder_warringer2003.py').read())

In [ ]:
name

In [ ]:
## pick wild-type curves to plot

list_to_plot = []
for name in df_wildtypes.index:
    if name[1]==1:
        list_to_plot.append(name)

list_to_plot = df_wildtypes.index

In [ ]:
### plot growth curve
fig, ax = plt.subplots(figsize = (0.7*FIGWIDTH_TRIPLET, 0.7*FIGHEIGHT_TRIPLET))

# plot original growth curves
for name in list_to_plot:

    t_trimmed, od_trimmed = get_excess_od_timeseries_before_trim(name)
    ax.plot(t_trimmed/60, od_trimmed, palette['wild-type'])
    
# plot mean growth curve
ax.plot(sol.t,sol.y[0], color = palette['wild-type median'], lw = 3, zorder = 3)

ax.set_yscale('log')
ax.set_xlabel('time [hours]')
ax.set_ylabel('absolute abundance [OD]')

ax.set_xlim(0,sol.t[-1])
ax.set_ylim(ymin,ymax)

fig.savefig(FIG_DIR + f"wildtype_averaged_with_raw_data.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

In [ ]:
df_knockouts.shape